# SIM - Declaração de Óbito (CID10) - Paraná 2022
Notebook validado e compatível com PySUS do AlertaDengue.

In [ ]:
!pip install git+https://github.com/AlertaDengue/PySUS.git --upgrade
# ⚠️ Após executar esta célula, reinicie a sessão do Google Colab dessa forma (Usando a interface do Google Colab):
# Em português: Vá em 'Tempo de execução' > 'Reiniciar sessão'
# Em inglês: Vá em 'Runtime' > 'Restart session'

In [1]:
# ⚠️ Após executar a célula acima, reinicie a sessão do Google Colab:
# Em português: Vá em 'Tempo de execução' > 'Reiniciar sessão'
# Em inglês: Vá em 'Runtime' > 'Restart session'
import pandas as pd
import matplotlib.pyplot as plt
from pysus.online_data.SIM import download

In [ ]:
# Baixar dados de óbitos (grupo CID10) do Paraná em 2022
df = download(states=["PR"], years=[2022], groups=["CID10"]).to_dataframe()
df.head()

In [ ]:
# Top 10 CIDs de causa básica de óbito
top_cids = df['CAUSABAS'].value_counts().head(10)
print(top_cids)

In [ ]:
# Gráfico de barras com os principais CIDs
top_cids.plot(kind='bar')
plt.title('Top 10 causas básicas de óbito - Paraná (2022)')
plt.xlabel('CID-10')
plt.ylabel('Número de óbitos')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de barras com os principais CIDs concatenados com as descrições
import requests
import xml.etree.ElementTree as ET

# URL do arquivo CID10.xml no GitHub
url = "https://raw.githubusercontent.com/cartaproale/PySUS/main/tabelas/CID10.xml"

# Baixar o arquivo XML diretamente do GitHub
response = requests.get(url)
response.encoding = 'ISO-8859-1'  # Garantir a codificação correta

# Corrigir caracteres especiais problemáticos
content = response.text.replace("&cruz;", "†")

# Analisar o conteúdo XML corrigido
root = ET.fromstring(content)

# Criar um dicionário para mapear códigos CID para suas descrições
cid_map = {}

# Iterar sobre as categorias para obter os códigos no formato LNN e LNNN
for category in root.findall(".//categoria"):
    code = category.get("codcat")
    description = category.find("nome").text

    # Verificar se o código está no formato LNN ou LNNN
    if code and description:
        # Concatenar o código original com a descrição
        cid_map[code] = f"{code} - {description}"

# Função para abreviar descrições longas e manter o código CID
def abbreviate_description(text, max_length=15):
    if len(text) > max_length:
        return text[:max_length] + "..."
    return text

# Verificar se há códigos não mapeados e fazer a correspondência correta
def map_cid_code(cid):
    # Tentar mapeamento direto
    if cid in cid_map:
        return abbreviate_description(cid_map[cid])
    # Tentar variantes do código (LNN, LNNN)
    if len(cid) == 4 and cid[:3] in cid_map:
        return abbreviate_description(f"{cid} - {cid_map[cid[:3]].split(' - ')[1]}")
    return cid  # Retorna o código se não encontrar

# Substituindo os códigos CID-10 pelas descrições no gráfico
top_cids_described = top_cids.rename(index=map_cid_code)

# Gráfico de barras com descrições dos CIDs
top_cids_described.plot(kind='bar')
plt.title('Top 10 causas básicas de óbito (descrições abreviadas) - Paraná (2022)')
plt.xlabel('Descrição do CID-10')
plt.ylabel('Número de óbitos')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
